In [10]:
# pip install sagemaker -U -i https://pypi.douban.com/simple

Looking in indexes: https://pypi.douban.com/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [4]:
# pip install -r requirements.txt -U -i https://pypi.douban.com/simple

In [5]:
config = dict()

config["S3_MODEL"] = "s3://hugging-face/llm/chatglm2-6b.tar.gz"
config['MODEL_NAME'] = "chatglm2-6b"

In [8]:
import boto3  
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
from time import gmtime, strftime

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

huggingface_model = HuggingFaceModel(
    model_data=config["S3_MODEL"],
    role=role,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    # instance_type='ml.m5.xlarge',
    instance_type='ml.g4dn.2xlarge',
    endpoint_name=config['MODEL_NAME'].upper() + strftime("-%Y%m%d-%H%M%S", gmtime()),
)

In [9]:
from sagemaker.huggingface.model import HuggingFacePredictor

predictor = HuggingFacePredictor(
  endpoint_name='CHATGLM2-6B-20230711-123646'
)

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [12]:
# send request
response = predictor.predict({"input": "七月份去京都，有什么推荐的景点或者活动吗？"})
print(response)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from model with message "Your invocation timed out while waiting for a response from container model. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://cn-northwest-1.console.aws.amazon.com/cloudwatch/home?region=cn-northwest-1#logEventViewer:group=/aws/sagemaker/Endpoints/CHATGLM2-6B-20230711-123646 in account 070507219079 for more information.